In [154]:
from pathlib import Path
from utils import read_input
from collections import Counter
import numpy as np
from bresenham import bresenham
from statistics import mean, median, mode
from math import factorial

In [155]:
test = Path("input/day8/test.txt")
test1 = Path("input/day8/test1.txt")
data = Path("input/day8/data.txt")

In [156]:
DISPLAY = {
    0: "abcefg",
    1: "cf",
    2: "acdeg",
    3: "acdfg",
    4: "bcdf",
    5: "abdfg",
    6: "abdefg",
    7: "acf",
    8: "abcdefg",
    9: "abcdfg"
}

In [157]:
display_counts = {k: len(v) for k, v in DISPLAY.items()}

In [158]:
display_counts

{0: 6, 1: 2, 2: 5, 3: 5, 4: 4, 5: 5, 6: 6, 7: 3, 8: 7, 9: 6}

In [159]:
unique_segment_counts = [k for k, v in Counter(display_counts.values()).items() if v == 1]

In [160]:
unique_segment_counts

[2, 4, 3, 7]

In [161]:
test_input = read_input(test)

In [162]:
test_input

['acedgfb cdfbe gcdfa fbcad dab cefabd cdfgeb eafb cagedb ab | cdfeb fcadb cdfeb cdbaf']

In [163]:
def _format(val):
    return tuple(val.strip().split(" "))
    
def get_entries(input_data):
    notes = read_input(input_data)
    entries = {}
    for line in notes:
        segment_patterns, output = line.split("|")
        segment_patterns = _format(segment_patterns)
        output = _format(output)
        assert segment_patterns not in entries
        entries[segment_patterns] = output
    return entries

In [164]:
entries = get_entries(data)

In [165]:
outputs = sum(entries.values(), ())

In [166]:
count_output_lengths = Counter([len(output) for output in outputs])

In [167]:
sum([v for k, v in count_output_lengths.items() if k in unique_segment_counts])

530

In [168]:
entries = get_entries(test)

In [169]:
DISPLAY_PATTERN = {
    0: "1110111",
    1: "0010010",
    2: "1011101",
    3: "1011011",
    4: "0111010",
    5: "1101011",
    6: "1101111",
    7: "1010010",
    8: "1111111",
    9: "1111011"
}

In [170]:
DISPLAY_COUNTS = {k: len(v) for k, v in DISPLAY.items()}

In [171]:
def find_unique(entry):
    found = {}
    for num in [1, 4, 7]:
        length = DISPLAY_COUNTS[num]
        segment = next(seg for seg in entry if len(seg) == length)
        found[num] = segment
    return found

In [172]:
def _set_positions(entry, known_mapping, position_map):
    # FIND POS 0
    # after finding the numbers with unique segment lengths, the one letter that appears only once is in pos 0
    position_map[0] = list(set(known_mapping[7]) - set(known_mapping[1]))[0]
    seen = set.union(*[set(val) for val in known_mapping.values()])
    
    pos_2_or_5 = set(known_mapping[1])
    pos_1_or_3 = set(known_mapping[4]) - pos_2_or_5
    
    # find 5 letter segments that contains one and only one letter that has not already been seen
    matches = [segment for segment in entry if len(segment) == 5 and len(set(segment) - seen) == 1]
    
    # FIND POS 2, 6, 5
    # Find the segment where the 3 letters that have already been seen and are not in the position map yet include both of pos_1_or_3
    
    known_positions = set([pos for pos in position_map if pos])  # should just be position 0 at this point
    
    for segment in matches:
        unseen = set(segment) - seen  # one letter
        segment_already_seen_and_unmapped = set(segment) - known_positions - unseen  # 3 characters
        
        # this segment contains 3 of the characters at position 2, 5, 1, or 3
        # find out if it contains only one of the characters at pos_2_or_6
        # If it does, this is pos 2, and this segment corresponds to 5
        
        found_2_or_5 = pos_2_or_5 - segment_already_seen_and_unmapped
        if found_2_or_5:
            # this segment is a 5
            known_mapping[5] = segment
            # the unseen letter is at position 6
            position_map[6] = list(unseen)[0]
            # the found letter is at position 2
            position_map[2] = list(found_2_or_5)[0]

            # we can identify the alternative position 5 too
            position_map[5] = list(pos_2_or_5 - found_2_or_5)[0]
    
    # FIND POS 1 and 3
    known_positions = set([letter for letter in position_map if letter])
    known_segments = list(known_mapping.values())
    matches = [segment for segment in entry if len(segment) == 5 and set(segment) not in known_segments]        

    for segment in matches:        
        unknown = set(segment) - set(known_positions)
        if len(unknown) == 1 and list(unknown)[0] in pos_1_or_3:
            known_mapping[3] = segment
            pos_3 = list(unknown)[0]
            pos_1 = list(pos_1_or_3 - unknown)[0]
            position_map[3] = pos_3
            position_map[1] = pos_1

            # remaining letter = position 4
            known_positions = set([letter for letter in position_map if letter])
            assert len(known_positions) == 6
            position_map[4] = list(set("abcdefg") - known_positions)[0]
    known_mapping[8] = next(seg for seg in entry if len(seg) == 7)

    return known_mapping, position_map

In [173]:
def _find_segment(number, position_map, entry):
    sorted_segment = ''.join(sorted([position_map[i] for i, val in enumerate(DISPLAY_PATTERN[number]) if int(val)]))
    for segment in entry:
        if ''.join(sorted(segment)) == sorted_segment:
            return segment

In [174]:
def _complete_map(known_mapping, position_map, entry):
    for key in DISPLAY_PATTERN.keys():
        if key not in known_mapping:
            known_mapping[key] = _find_segment(key, position_map, entry)
    return known_mapping

In [175]:
def decode(entry, outputs):
    position_map = [''] * 7
    known_mapping = find_unique(entry)
    known_mapping, position_map = _set_positions1(entry, known_mapping, position_map)
    known_mapping = _complete_map(known_mapping, position_map, entry)
    known_numbers = {tuple(sorted(v)): k for k, v in known_mapping.items()}
    
    return int(''.join([str(known_numbers[tuple(sorted(output))]) for output in outputs]))
    

In [176]:
entries = get_entries(data)

In [177]:
entry = list(entries.keys())[0]
    
known_mapping = find_unique(entry)

In [178]:
sum(decode(k, v) for k, v in entries.items())

1051087